We adjust the number of input nodes to reflect the number of datapoints in each trainning example.

In [21]:
import struct as st
import numpy as np

filename = {'images' : 'mnist/train/train-images-idx3-ubyte',
            'labels' : 'mnist/train/train-labels-idx1-ubyte'}

In [22]:
def read_train_set_images(filename): 
    images_file = open(filename, 'rb');
    images_file.seek(0);
    magic = st.unpack('>4B', images_file.read(4));
    number_images = st.unpack('>I', images_file.read(4))[0];
    number_rows = st.unpack('>I', images_file.read(4))[0];
    number_columns = st.unpack('>I', images_file.read(4))[0];
    #print('Rows: ' + str(number_rows) + 'Columns: ' + str(number_columns));
    n_bytes =  1000 * (number_rows * number_columns);
    train_array_images = 255 - np.asarray(st.unpack('>' + 'B' * n_bytes, 
                                    images_file.read(n_bytes))).reshape((1000, number_rows, number_columns));
    print('Images: ' + str(train_array_images.shape));
    return train_array_images;

def read_train_set_labels(filename):
    labels_file = open(filename, 'rb');
    labels_file.seek(0);
    magic_number = st.unpack('>4B', labels_file.read(4));
    number_items = st.unpack('>I', labels_file.read(4))[0];
    n_bytes = 1000;
    train_array_labels =  9 - np.asarray(st.unpack('>' + 'B' * n_bytes,
                                            labels_file.read(n_bytes)))
    print('Labels: ' + str(train_array_labels.shape));
    return train_array_labels;

In [23]:
images_array = read_train_set_images(filename['images']);
labels_array = read_train_set_labels(filename['labels']);
print(labels_array[0]);

Images: (1000, 28, 28)
Labels: (1000,)
4


In [24]:
#flatten example
zeros = np.zeros((4, 5, 6, 7, 8));
zeros = zeros.reshape((*zeros.shape[:1], -1, *zeros.shape[-1:]));
print(zeros.shape);
# np.reshape(()) can take a -1 as an argument, meaning 'total array
# size divided by product of all other listed dimensions'.

(4, 210, 8)


In [25]:
#flatten images array;
images_array = images_array.reshape((*images_array.shape[:1], -1));
print(images_array.shape);

(1000, 784)


One output node trainning => it wouldn't actually work for we would update the same set weights, therefore the last number(image) predicted would set the weights to recognize those specific patterns. 

One output node for each of our numbers i.e. 0 to 9;

In [44]:
weights = np.random.rand(10, 784).T;
alpha = .1;

def wrap_goal_pred(labels_array):
    output = {};
    for i in range(len(labels_array)):
        output[i] = goal_pred_by_label(labels_array[i]);
    print(output.shape);
    
def goal_pred_by_label(label):
    return {
         0: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         1: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         2: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
         3: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
         4: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
         5: [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
         6: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
         7: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
         8: [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
         9: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
    }.get(label);

def neural_network(input, weights):
    pred = np.dot(input, weights);
    msquared = np.subtract(pred, goal_pred) ** 2;
    
wrap_goal_pred(labels_array);
neural_network(images_array, weights);

AttributeError: 'dict' object has no attribute 'shape'